<a href="https://www.kaggle.com/code/eounal/random-forest-maras?scriptVersionId=127755353" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Random Forest Analysis For Kahramanmaraş Earthquake**

****

In [ ]:
import glob
import tifffile as tf
import numpy as np
import time
import functions

start_time = time.time()

MainFolder = "../.."
Features = glob.glob(MainFolder + "/Features/**/*.tif" , recursive=True) # include subfolders recursive=True
InventoryFile= MainFolder + "/Inventory/Inventory.tif"
DEMFile = MainFolder + "/Features/DEM/Altitude.tif"
# Training Parameters
InventoryRate = 1
NonLSRate = 2
TestRate = 0.2
# Classification Parameters
n_estimators = 34
max_depth = 8
# Versioning
DemVersion = "EU25m"
ResultVersion= f"{DemVersion}-{InventoryRate}ls-{NonLSRate}nonls-{n_estimators}est-{max_depth}mxdpth"
ResultFile = MainFolder + f"/Result/Result_{ResultVersion}"

In [ ]:
X_train, X_test, y_train, y_test = functions.FeatureSet.CreateTrainAndTestData(Features, 
                                                                               InventoryFile, 
                                                                               InventoryRate, 
                                                                               TestRate, 
                                                                               NonLSRate)
#np.savez_compressed('Xy_train.npz',x=X_train,y=y_train)

In [ ]:
args = {
    "n_estimators": n_estimators,
    "criterion": "entropy",
    "max_depth": max_depth,
    "min_samples_split": 2,
    "min_samples_leaf": 2,
    "class_weight": "balanced",
    "bootstrap": True,
    "random_state": 32,
    "oob_score": True,
    "n_jobs": -1,
    "max_features": "sqrt",
}
RFClassifier, y_pred = functions.FeatureSet.RandomForestClassification(X_train, 
                                                                       y_train.ravel(), 
                                                                       X_test, 
                                                                       args)

In [ ]:
import joblib
# Save model to file
joblib.dump(RFClassifier, f'../Model/RF_LS_Model_{ResultVersion}.pkl')
# Load model from file
# RFClassifier = joblib.load(MainFolder + '/Study/Model/RF_LS_Model.pkl')

In [ ]:
functions.FeatureSet.ShowImportances(f'{MainFolder}/Figures', 
                                     ResultVersion, 
                                     Features, 
                                     RFClassifier)

In [ ]:
functions.FeatureSet.ShowMetrics(f'{MainFolder}/Figures', 
                                 ResultVersion, 
                                 RFClassifier, 
                                 X_test, 
                                 y_test, 
                                 y_pred)

In [ ]:
# regr = RandomForestRegressor(max_depth=2, random_state=0)
# regr.fit(X_train_s, y_train_s.ravel())
# y_pred_s=regr.predict(X_test_s)

In [ ]:
Classes = [] 
for image in Features:
    cache = tf.imread(image).flatten()
    cache[np.isnan(cache)]=0
    Classes.append(cache.T)
Classes = np.asarray(Classes)

In [ ]:
#import joblib
# Load model from file
#RFClassifierModel = joblib.load('/kaggle/working/RF_LS_Model.pkl')

In [ ]:
#from numba import jit, cuda
#@jit(target_backend='cuda')
def PredictClassProbabilities(Classifier):
    classProbabilities = Classifier.predict_proba(Classes.T)[:,1]
    return classProbabilities
ClassProbabilities = PredictClassProbabilities(RFClassifier)

In [ ]:
#cols, rows = functions.FeatureSet.Dimensions(DEMFile)
# no_datas = np.where(np.isnan(tf.imread(DEMFile).flatten())) 
no_datas = np.where(tf.imread(DEMFile).flatten()<0)
ClassProbabilities[no_datas]=0
#data = ClassProbabilities.reshape(rows,cols)
data = ClassProbabilities.reshape(12839,6880)

import matplotlib.pyplot as plt
plt.style.use('default')
plt.imshow(data, cmap=plt.cm.viridis)
plt.colorbar()
plt.savefig("RF-ROC_map", dpi = 600)

In [ ]:
#functions.FeatureSet.GeoExport(data, ResultFile, DEMFile, -9999)

In [ ]:
def PrintElapsedTime(start_time):
    from datetime import timedelta
    elapsed = time.time() - start_time
    td_str = str(timedelta(seconds=elapsed))
    x = td_str.split(':')
    print(f'{x[0]} saat {x[1]} dakika {x[2]} saniyede tamamlandı.')
PrintElapsedTime(start_time)